# Import Libraries

In [1]:
import re
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

### Link of Pages

In [2]:
#################################################################  Number of pages to scrap ####################################################################################
num_pages_to_scrap = 225
####################################  ##########################  Link of Second page as Example ###############################################################################

link_page_2_as_example = 'https://www.amazon.com/s?i=computers&rh=n%3A565108&fs=true&page=2&qid=1645999091&ref=sr_pg_2'
###################################################################  Links of pages #################################################################################

pattern_of_urls = re.sub(r'&page=\d','&page={}',str(link_page_2_as_example))
pattern_of_urls = re.sub(r'&ref=sr_pg_\d','',str(pattern_of_urls))

links = [
    pattern_of_urls.format(page_num) for page_num in range(1, num_pages_to_scrap+1)
        ]

In [3]:
###################################################################  Start Scraping ###########################################################################################
####################################################  Prevent to load pictures(because we don't need them) to scrap faster ####################################################

chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

###############################################################################################################################################################################

driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get(links[0])
driver.maximize_window()

lst_of_products = []
num_current_page = 1
print('You are in page: ' , num_current_page)
while num_current_page <= num_pages_to_scrap :
    if (num_current_page % 15 == 0):
        print('You are in page: ' , num_current_page)
    driver.get(links[num_current_page-1])
    divs_item_on_page = driver.find_elements(by=By.CSS_SELECTOR, value='.s-card-border')
    for div_item in divs_item_on_page:
        current_dict = {}
        ActionChains(driver) \
            .key_down(Keys.CONTROL) \
            .click(div_item) \
            .key_up(Keys.CONTROL) \
            .perform()
        driver.switch_to.window(driver.window_handles[-1])
        #time.sleep(1)
################################################### now You are in New tab!(Product Tab) #######################################################################
        see_more_btn = driver.find_elements(by=By.CSS_SELECTOR, value='#poToggleButton .a-expander-prompt')
        if len(see_more_btn) != 0:
            see_more_btn[0].click()

##############################################################################  Title  #############################################################################
        title_value = driver.find_elements(by=By.CSS_SELECTOR, value='#productTitle')
        if len(title_value) != 0 :
            current_dict.update({'title':title_value[0].text})

########################################################################Info in the table on the top#################################################################
        top_table = driver.find_elements(by=By.CSS_SELECTOR, value='.a-normal')
        if len(top_table) != 0:
            top_table_rows = top_table[0].find_elements(by=By.CSS_SELECTOR,value='tr')
            for row in top_table_rows:
                row_title_value = row.find_elements(by=By.CSS_SELECTOR, value='td')
                row_title = row_title_value[0].text.lower().replace(' ', '_')
                row_info = row_title_value[1].text
                current_dict.update({row_title: row_info})

####################################################Info in the table on the bottom of the page#####################################################################
        bottom_table_rows = driver.find_elements(by=By.CSS_SELECTOR, value='#productDetails_feature_div tr')
        if len(bottom_table_rows) != 0:
            for row in bottom_table_rows:
                row_title = row.find_elements(by=By.CSS_SELECTOR, value='th')[0]
                row_value = row.find_elements(by=By.CSS_SELECTOR, value='td')[0]
                row_title = row_title.text.lower().replace(' ', '_')
                row_info = row_value.text
                current_dict.update({row_title: row_info})

############################################## Data In Comparison Table (Fetch First Column Info's)##################################################################
        comparison_rows = driver.find_elements(by=By.CSS_SELECTOR, value='#HLCXComparisonWidget_feature_div table tr')
        if len(comparison_rows) != 0:
            desired_rows = comparison_rows[4:]
            for row in desired_rows:
                row_title = row.find_elements(by=By.CSS_SELECTOR, value='th')[0]
                row_value = row.find_elements(by=By.CSS_SELECTOR, value='td')[0]
                row_title = row_title.text.lower().replace(' ', '_')
                row_info = row_value.text
                current_dict.update({row_title: row_info})

############################################################################      Stars           ##################################################################
        stars = driver.find_elements(by=By.XPATH, value='//div[@class="a-section a-spacing-none a-spacing-top-mini cr-widget-ACR"]')
        if len(stars)!= 0:
            current_dict.update({'stars': stars[0].text[:3]})
############################################################################      Price           ##################################################################
        price = driver.find_elements(by=By.CSS_SELECTOR, value='#corePrice_feature_div span')
        if len(price)!= 0:
            current_dict.update({'price': price[-1].text})
######################################## End of Scraping Product page!Now close the Product page and Switch to Main page  ##########################################
        lst_of_products.append(current_dict)
        driver.execute_script("window.close()")
        driver.switch_to.window(driver.window_handles[0])

################################################## prepare next URL(next page)  #################################################################################
    num_current_page = num_current_page + 1
###################################################  End of Scraping  #######################################################################################

C:\Users\Farshad\AppData\Local\Temp/ipykernel_6256/1679463407.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


You are in page:  1
You are in page:  15
You are in page:  30
You are in page:  45
You are in page:  60
You are in page:  75
You are in page:  90
You are in page:  105
You are in page:  120
You are in page:  135
You are in page:  150
You are in page:  165
You are in page:  180
You are in page:  195
You are in page:  210
You are in page:  225


In [98]:
###########################################  Unique Feature names of products  ##############################################################################
all_features_for_all_products = []
for product in lst_of_products:
    current_product_features = list(product.keys())
    all_features_for_all_products.extend(current_product_features)
    all_features_for_all_products = list(set(all_features_for_all_products))
    all_features_for_all_products.sort()

######################################################## Create Numpy Matrix of string #####################################################################
number_of_products = len(lst_of_products)
number_of_columns = len(all_features_for_all_products)
numpy_table = np.empty((number_of_products,number_of_columns,), dtype=object)
numpy_table[:] = np.nan

for product_row,item in enumerate(lst_of_products):
    for key in item:
        col_number = all_features_for_all_products.index(key)
        numpy_table[product_row,col_number] = item[key]


####################################################  Create Pandas DataFrame ################################################################################
data = pd.DataFrame(numpy_table,columns=all_features_for_all_products)
data.head()

,,asin,audio-out_ports_(#),average_battery_life_(in_hours),batteries,batteries_included?,batteries_required?,battery_cell_composition,battery_cell_type,best_sellers_rank,...,tech_spec_battery_description_toys,title,total_usb_ports,usage,voltage,warranty_description,wireless_communication_standard,wireless_communication_technology,wireless_type,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,B09HVC79PC,NaN,10 Hours,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN,#23 in Computers & Accessories (See Top 100 in...,...,NaN,"Acer Aspire 5 A515-46-R3UB | 15.6"" Full HD IPS...",NaN,NaN,NaN,NaN,Bluetooth,NaN,Bluetooth,NaN
3,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"2021 Apple MacBook Pro (14-inch, Apple M1 Pro ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,B09BG96KFJ,NaN,12 Hours,1 Lithium Polymer batteries required. (included),NaN,NaN,NaN,NaN,#31 in Computers & Accessories (See Top 100 in...,...,NaN,"Lenovo Flex 5 Laptop, 14.0"" FHD Touch Display,...",NaN,NaN,NaN,NaN,802.11ac,NaN,802.11ac,NaN


##### Primary Table Shape

In [99]:
print(f'number of rows in DataFrame is {data.shape[0]} and numer of columns is {data.shape[1]}')

number of rows in DataFrame is 5388 and numer of columns is 120


##### as we see, this code was not only for laptop :))
##### now we have our table, and after **Data Cleaning**, we could do some stuff like Predictive analytics and Descriptive analytics

# Some General Data Cleaning Steps

#### 1.Replace empty string ('') cells with NaN

In [100]:
new_df = data.replace(r'^\s*$', np.NaN, regex=True)

#### 2.Drop Rows which all their values are NaN

In [101]:
new_df.dropna(how='all',inplace=True)

#### 3.Drop columns which more than 50% of them is Null

In [102]:
new_df = new_df.loc[:, new_df.isnull().mean() < 0.5]

#### 4.Drop rows which more than 50% of them is Null

In [103]:
new_df = new_df[new_df.isnull().mean(axis=1) < 0.5]

#### 5.Remove Duplicate rows

In [104]:
new_df = new_df.drop_duplicates()

## Secondary Table

In [105]:
new_df.head(10)

,asin,best_sellers_rank,brand,card_description,chipset_brand,color,computer_memory_size,computer_memory_type,cpu_model_manufacturer,cpu_speed,...,ram,ram_type,screen_size,series,sold_by,standing_screen_display_size,stars,title,wireless_communication_standard,wireless_type
2,B09HVC79PC,#23 in Computers & Accessories (See Top 100 in...,Acer,Integrated,AMD,Silver,4 GB,DDR4 SDRAM,AMD,3.5 GHz,...,4 GB DDR4,DDR4 SDRAM,15.6 inches,A515-46-R3UB,ChiefComputers,15.6 Inches,4.6,"Acer Aspire 5 A515-46-R3UB | 15.6"" Full HD IPS...",Bluetooth,Bluetooth
4,B09BG96KFJ,#31 in Computers & Accessories (See Top 100 in...,Lenovo,Integrated,AMD,Graphite Grey,16 GB,DDR4 SDRAM,AMD,2.1 GHz,...,16 GB DDR4,DDR4 SDRAM,14 inches,Ideapad Flex 5 14 Laptop,Best Electronics (SN Recorded),14 Inches,4.5,"Lenovo Flex 5 Laptop, 14.0"" FHD Touch Display,...",802.11ac,802.11ac
5,B09FXFDGN3,#72 in Computers & Accessories (See Top 100 in...,HP,Integrated,Intel,Natural silver,8.0 GB,DDR4 SDRAM,Intel,4.2 GHz,...,8 GB DDR4,DDR4 SDRAM,15.6 inches,HP 15 Laptop,ebestpurchase,15.6 Inches,4.4,"HP 15-inch Laptop, 11th Generation Intel Core ...","Bluetooth, 802.11ax","Bluetooth, 802.11ax"
6,B092YHJLS6,#189 in Computers & Accessories (See Top 100 i...,Acer,RTX 3060,NVIDIA,Black,16 GB,DDR4 SDRAM,Intel,4.6 GHz,...,16 GB DDR4,DDR4 SDRAM,15.6 inches,PH315-54-760S,Midland Electronics LLC,15.6 Inches,4.6,Acer Predator Helios 300 PH315-54-760S Gaming ...,802.11ax,802.11ax
7,B09G8SK2KK,#275 in Computers & Accessories (See Top 100 i...,HP,Integrated,AMD,Natural silver,8 GB,DDR4 SDRAM,AMD,4 GHz,...,8 GB DDR4,DDR4 SDRAM,14 inches,HP Laptop 14-fq1025nr,Tech Solution (SN Recorded),14 Inches,4.5,"HP 14 Laptop, AMD Ryzen 5 5500U, 8 GB RAM, 256...","802.11ax, Bluetooth","Bluetooth, 802.11ax"
8,B09FX1YF28,#345 in Computers & Accessories (See Top 100 i...,HP,Integrated,Intel Iris Xe Graphics,Natural silver aluminum,16 GB,DDR4 SDRAM,Intel,4.7 GHz,...,16 GB DDR4,DDR4 SDRAM,15.6 inches,HP Pavilion Laptop 15-eg0025nr,GreatPriceTech (Expedite Shipping) (Record S/N),15.6 Inches,4.5,"HP Pavilion 15 Laptop, 11th Gen Intel Core i7-...","Bluetooth, 802.11ax","Bluetooth, 802.11ax"
9,B09127DDVT,#525 in Computers & Accessories (See Top 100 i...,MSI,Dedicated,NVIDIA,Core Black,16 GB,NaN,Intel,2.4 GHz,...,16 GB DDR4,NaN,15.6 inches,Pulse GL66 11UGK-001,ElectroTechOutlet,15.6 Inches,4.5,"MSI GL66 Gaming Laptop: 15.6"" 144Hz FHD 1080p ...",Bluetooth,Bluetooth
10,B09BDGD2TZ,#457 in Computers & Accessories (See Top 100 i...,Lenovo,Dedicated,NVIDIA,Phantom Blue,16 GB,NaN,AMD,3.2,...,16 GB DDR4,NaN,15.6,Legion 5 15,antonline,15.6,4.6,"Lenovo Legion 5 Gaming Laptop, 15.6"" FHD Displ...",802.11ax,802.11ax
11,B09HX29TPY,#261 in Computers & Accessories (See Top 100 i...,ASUS,Integrated,Intel,Star Black,4 GB,DDR4 SDRAM,Intel,2.8 GHz,...,4 GB DDR4,DDR4 SDRAM,11.6 inches,ASUS Laptop L210,Cardinal Pro Electronics,11.6 Inches,3.9,"ASUS Laptop L210 11.6” Ultra Thin, Intel Celer...","Bluetooth, 802.11ac","Bluetooth, 802.11ac"
12,B08XPC3WFQ,#640 in Computers & Accessories (See Top 100 i...,ASUS,GeForce RTX 3050 Ti,NVIDIA,Eclipse Grey,8 GB,DDR4 SDRAM,Intel,4.8 GHz,...,8 GB DDR4,DDR4 SDRAM,15.6 inches,ASUS TUF Dash,BatzmaruEbiz,15.6 Inches,4.5,ASUS TUF Dash 15 (2021) Ultra Slim Gaming Lapt...,"802.11ax, Bluetooth","Bluetooth, 802.11ax"


##### Secondary table shape

In [106]:
print(f'Number of rows in DataFrame is {new_df.shape[0]} and numer of columns is {new_df.shape[1]}')

Number of rows in DataFrame is 4568 and numer of columns is 42


#### These Steps were General steps for Data Cleaning
#### For specific Category of Data(for example Laptop,Cellphone,Headphone and ...) we could apply more specific Data cleaning methods(base on the product columns)
<br/>
<hr/>

##### Now ,as an Example , we want to find the rows(products) which their Title contains 'Nitro 5'

In [107]:
new_df.loc[new_df['title'].str.contains('Nitro 5'), :]

,asin,best_sellers_rank,brand,card_description,chipset_brand,color,computer_memory_size,computer_memory_type,cpu_model_manufacturer,cpu_speed,...,ram,ram_type,screen_size,series,sold_by,standing_screen_display_size,stars,title,wireless_communication_standard,wireless_type
170,B08YZ6Z1G3,"#3,228 in Computers & Accessories (See Top 100...",Acer,Dedicated,NVIDIA,Black,16 GB,NaN,AMD,4.4 GHz,...,16 GB DDR4,NaN,15.6 inches,AN515-45-R92M,Midland Electronics LLC,15.6 Inches,4 o,"Acer Nitro 5 AN515-45-R92M Gaming Laptop, AMD ...",Bluetooth,Bluetooth
505,B08YZ52KK1,"#23,706 in Computers & Accessories (See Top 10...",Acer,Dedicated,NVIDIA,Black,32 GB,NaN,AMD,4.6 GHz,...,32 GB DDR4,NaN,15.6 inches,AN515-45-R9QH,"J-Tech Digital, Inc",15.6 Inches,4.7,"Acer Nitro 5 AN515-45-R9QH Gaming Laptop, AMD ...",Bluetooth,Bluetooth
575,B08YZ1V9WP,"#9,174 in Computers & Accessories (See Top 100...",Acer,Dedicated,NVIDIA,Black,16 GB,DDR4 SDRAM,AMD,4.4 GHz,...,16 GB DDR4,DDR4 SDRAM,17.3 inches,AN517-41-R0RZ,High Tech Stop,17.3 Inches,4.5,"Acer Nitro 5 AN517-41-R0RZ Gaming Laptop, AMD ...",Bluetooth,Bluetooth
832,B08C1LB2S9,"#15,583 in Computers & Accessories (See Top 10...",Acer,Dedicated,NVIDIA,Black,8 GB,DDR4 SDRAM,Intel,4.10 GHz,...,8 GB DDR4,DDR4 SDRAM,15.6 inches,AN515-54-5812,Cardinal Pro Electronics,15.6 Inches,4.7,"Acer Nitro 5 Gaming Laptop, 9th Gen Intel Core...",5 GHz Radio Frequency,5 GHz Radio Frequency
1022,B09B2SP417,"#15,343 in Computers & Accessories (See Top 10...",Acer,Dedicated,NVIDIA,NaN,16.0 GB,DDR4 SDRAM,Intel,4.50 GHz,...,16 GB DDR4,DDR4 SDRAM,15.6 inches,Acer Nitro,Available from these sellers,15.6 Inches,4.3,"Newest Acer Nitro 5 Gaming Laptop, NVIDIA GeFo...",NaN,NaN
1026,B08J3XSXQ5,"#14,263 in Computers & Accessories (See Top 10...",Acer,Dedicated,NVIDIA,NaN,8 GB,DDR4 SDRAM,AMD,2.40 GHz,...,8 GB DDR4,DDR4 SDRAM,15.6 inches,NaN,VINE EXPRESS STORE,15.6 Inches,4.6,"Acer Nitro 5 Gaming Laptop, 9th Gen Intel Core...",802.11ac,802.11ac
1245,B09GVTFM4Q,"#13,220 in Computers & Accessories (See Top 10...",Acer,Dedicated,NVIDIA,RTX 3050 | 16GB | 512GB,16 GB,DDR4 SDRAM,Intel,2.50 GHz,...,16 GB DDR3,DDR4 SDRAM,15.6 inches,Nitro 5,Available from these sellers,15.6 Inches,4.2,Acer Nitro 5 AN515 Gaming Laptop | Intel Core ...,802.11a/b/g/n/ac,802.11a/b/g/n/ac
1615,B09ST8HR27,"#29,871 in Computers & Accessories (See Top 10...",Acer,Dedicated,NVIDIA,NaN,NaN,DDR4 SDRAM,NaN,NaN,...,16 GB DDR4,NaN,NaN,Acer Nitro,NaN,15.6 Inches,4.8,Acer Nitro 5 Gaming Laptop 15.6” FHD IPS Displ...,NaN,Bluetooth
1962,B087RSQGSV,"#30,877 in Computers & Accessories (See Top 10...",Acer,Dedicated,NVIDIA,NaN,16 GB,DDR4 SDRAM,Intel,4.5 GHz,...,16 GB DDR4,DDR4 SDRAM,17.3 inches,AN517-51-76V6,Fulton Tech (We Record SN),17.3 Inches,4.6,"Acer Nitro 5 Gaming Laptop, 9th Gen Intel Core...",801.11ac,801.11ac
3216,B08JKYY74G,"#74,376 in Computers & Accessories (See Top 10...",Acer,Dedicated,NVIDIA,Obsidian Black,8 GB,SODIMM,Intel,4.50 GHz,...,8 GB SO-DIMM,SODIMM,17.3 inches,Nitro 5,Prime Service For Less,17.3 Inches,4.5,"Acer - Nitro 5 17.3"" Gaming Laptop - Intel Cor...",802.11ac,802.11ac


# Export Dataframe as Excell file

In [108]:
new_df.to_excel('laptop_amazon.xlsx')